In [1]:
import time
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random

In [2]:
ol = Overlay('/home/xilinx/jupyter_notebooks/smul/smul.bit') # check your path
ol.download() # it downloads your bit to FPGA
dma = ol.smul_dma # creating a dma instance. Note that we packed smul and smul_dma into streamMul
sadd_ip = MMIO(0x40000000, 0x10000) # we got this IP from Address Editor
xlnk = Xlnk()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: pynq.Xlnk is deprecated and will be removed in 2.7 - use pynq.allocate instead
  """


In [3]:
length = 11

in_buffer = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer
out_buffer = xlnk.cma_array(shape=(length,), dtype=np.int32) # output buffer

samples = random.sample(range(0, length), length)
np.copyto(in_buffer, samples) # copy samples to inout buffer

sadd_ip.write(0x10, length) # we got this address from Vivado source
t_start = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait() # wait for send channel
dma.recvchannel.wait() # wait for recv channel
t_stop = time.time()
in_buffer.close()
out_buffer.close()
print('Hardware execution time: ', t_stop-t_start)
for i in range(0, length):
    print('{}*2 = {}'.format(in_buffer[i], out_buffer[i]))

Hardware execution time:  0.0025377273559570312
9*2 = 18
4*2 = 0
0*2 = 0
8*2 = 0
6*2 = 12
5*2 = 0
10*2 = 20
1*2 = 0
2*2 = 4
7*2 = 0
3*2 = 6
